In [107]:
import numpy as np
import pandas as pd

In [106]:
class Game:
    def __init__(self):
        self.deck = []
        self.hand = []
        self.dealer_hand = []
        self.hand_history = []
        
    def get_deck(self):
        if len(self.deck) == 0:
            deck = np.array([])
            for i in range(1,10):
                deck = np.concatenate((deck, np.full(4,i)))
            return np.concatenate((deck, np.full(16,10)))
        else:
            return self.deck
        
    def set_deck(self, deck):
        self.deck = deck
        
    def get_dealer_hand(self):
        return self.dealer_hand
    
    def set_dealer_hand(self, hand):
        self.dealer_hand = hand
        
    def get_hand(self):
        return self.hand
    
    def set_hand(self, hand):
        self.hand = hand

    def get_hand_history(self):
        return self.hand_history
    
    def add_hand_history(self, obj):
        self.hand_history = self.hand_history + [obj]
        
    def add_card(self, deck):
            card = deck[np.random.randint(len(deck))]
            deck = np.delete(deck, np.where(deck == card)[0][0])
            return card, deck
    
    def evaluate_hand(self, hand):
        if 1 in hand:
            if 10 in hand and len(hand) == 2:
                return [21]
            return sum(hand) + 10 if sum(hand) + 10 <= 21 else sum(hand)
        return sum(hand)
            
    def dealer_turn(self, hand, deck, p_score):
        score = self.evaluate_hand(hand)
        if score <= p_score and score < 17:
            card, deck = self.add_card(deck)
            hand = np.concatenate([hand, [card]])
            return self.dealer_turn(hand, deck, p_score)
        else:
            return hand, score, deck
        
    def return_option(self, options):
        return options[np.random.randint(len(options))]
    
    def return_result(self, outcome, dealer_hand, player_hand, double):
        return {
            "outcome" : outcome,
            "dealer_hand" : dealer_hand,
            "player_hand" : player_hand,
            "double": double
        }
        
    def start_game(self, bet, num_decks):
        self.set_deck(np.concatenate(np.array([self.get_deck()]*num_decks)))
        deck = self.get_deck()
        dealer = []
        player = []
        count = 0
        while(len(player)<2 or len(dealer)<2):
            card = deck[np.random.randint(len(deck))]
            deck = np.delete(deck, np.where(deck == card)[0][0])
            player.append(card) if count % 2 == 0 else dealer.append(card)
            count+=1
        self.set_dealer_hand(dealer)
        self.set_hand(player)
        return self.play(bet)
    
    def play(self, bet):
        dealer_hand = self.get_dealer_hand()
        player_hand = self.get_hand()
        dealer_score = self.evaluate_hand(dealer_hand)
        player_score = self.evaluate_hand(player_hand)
        results = {}
        if dealer_score == [21]:
            if player_score == [21]:
                self.return_result("push", dealer_hand, player_hand, False)
            else:
                results = self.return_result("loss", dealer_hand, player_hand, False) 
        if player_score == [21]:
            results = self.return_result("win", dealer_hand, player_hand, False) 
        if player_score != [21] and dealer_score != [21]:
            def recurse(hand, dealer_score, dealer_hand, first):
                score = self.evaluate_hand(hand)
                deck = self.get_deck()
                if score > 17 and 1 not in hand:
                    action = "s"
                elif first and hand[0] == hand[1]:
                    action = self.return_option(['p'])
                else:   
                    action = self.return_option(['h','s','d']) if first else self.return_option(['h','s'])
                if action == "p":
                    return self.split(bet)    
                double = action == "d"
                if action == "h" or action == "d":
                    card, deck = self.add_card(deck)
                    hand += [card]
                    self.set_deck(deck)
                    if action == "h":
                        bust_check = self.evaluate_hand(hand)
                        if bust_check > 21:
                            return self.return_result("loss", dealer_hand, player_hand, double) 
                        else:
                            return recurse(hand, dealer_score, dealer_hand, False)
                hand_score = self.evaluate_hand(hand)
                dealer_hand, dealer_score, deck = self.dealer_turn(dealer_hand, deck, hand_score)
                if (dealer_score > hand_score and dealer_score<=21) or hand_score > 21:
                    return self.return_result("loss", dealer_hand, player_hand, double)
                elif dealer_score == hand_score:
                    return self.return_result("push", dealer_hand, player_hand, double)
                else:
                    return self.return_result("win", dealer_hand, player_hand, double)
            results = recurse(player_hand, dealer_score, dealer_hand, True)         
        self.add_hand_history(results)
        self.set_hand([])
        self.set_dealer_hand([])
        return results 
        
    def split(self, bet):
        def split_recurse(hand, dealer_hand, first):
            score = self.evaluate_hand(hand)
            deck = self.get_deck()
            dealer_score = self.evaluate_hand(dealer_hand)
            if score == [21]:
                return hand
            if score > 17 and 1 not in hand:
                action = "s"
            if first and hand[0] == hand[1]:
                action = self.return_option(['h','s','d', 'p'])
            else:   
                action = self.return_option(['h','s','d']) if first else self.return_option(['h','s'])
            if action == "p":
                self.set_hand(self.get_hand() + [hand[1]])
                return self.split(bet)
            if action == "h" or action == "d":
                card, deck = self.add_card(deck)
                hand += [card]
                self.set_deck(deck)
                if action == "h":
                    bust_check = self.evaluate_hand(hand)
                    if bust_check > 21:
                        return hand
                    else:
                        return split_recurse(hand, dealer_hand, False)
                if action == "d":
                    return [hand]
            if action == "s":
                return hand
        results = []
        deck = self.get_deck()
        player_hand = self.get_hand()
        dealer_hand = self.get_dealer_hand()
        for card in player_hand:
            new_card, deck = self.add_card(deck)
            hand = [card, new_card]
            results.append(split_recurse(hand, dealer_hand, True))
        doubles = [type(x[0]) == list for x in results]
        results = [x[0] if type(x[0]) == list else x for x in results]
        scores = [self.evaluate_hand(x) for x in results]
        max_score = max([0 if x > 21 else x for x in [0 if x == [21] else x for x in scores]])
        dealer_hand, dealer_score, deck = self.dealer_turn(dealer_hand, deck, max_score)
        hand_data = {}
        hand_count = 0
        for hand_score, player_hand, double in zip(scores, results, doubles):
            if hand_score == [21]:
                hand_data.update({
                    hand_count: self.return_result("win", dealer_hand, player_hand, double)
                    })
            elif (dealer_score > hand_score and dealer_score<=21) or hand_score>21:
                hand_data.update({hand_count: self.return_result("loss", dealer_hand, player_hand, double)}) 
            elif dealer_score == hand_score:
                hand_data.update({hand_count: self.return_result("push", dealer_hand, player_hand, double)})
            else:
                hand_data.update({hand_count: self.return_result("win", dealer_hand, player_hand, double)})
            hand_count+=1
        self.add_hand_history(hand_data)
        self.set_hand([])
        self.set_dealer_hand([])
        return hand_data        

In [89]:
observations = []

In [90]:
for i in range(200):
    game = Game()
    for i in range(10):
        observations.append(game.start_game(10,1))

In [105]:
df = pd.DataFrame(list(filter(lambda x: len(x)!=2, observations)))
df["dealer_score"] = df.dealer_hand.apply(lambda x: game.evaluate_hand(x))
df["player_score"] = df.player_hand.apply(lambda x: game.evaluate_hand(x))
df["d_blackjack"] = df.dealer_score.apply(lambda x: True if x == [21] and type(x) == list else False)
df["p_blackjack"] = df.player_score.apply(lambda x: True if x == [21] and type(x) == list else False)
df["dealer_card"] = df.dealer_hand.apply(lambda x: x[0])
df["splitable"] = df.player_hand.apply(lambda x: True if x[0] == x[1] else False)
df

,outcome,dealer_hand,player_hand,double,dealer_score,player_score,d_blackjack,p_blackjack,dealer_card,splitable
0,loss,"[10.0, 1.0]","[8.0, 10.0]",False,[21],18,True,False,10.0,False
1,win,"[5.0, 10.0, 10.0]","[10.0, 10.0]",False,25,20,False,False,5.0,True
2,win,"[10.0, 2.0, 6.0]","[9.0, 10.0]",False,18,19,False,False,10.0,False
3,loss,"[9.0, 2.0, 10.0]","[8.0, 9.0, 3.0]",True,21,20,False,False,9.0,False
4,loss,"[10.0, 2.0, 8.0]","[5.0, 8.0]",False,20,13,False,False,10.0,False
...,...,...,...,...,...,...,...,...,...,...
1928,loss,"[10.0, 3.0, 10.0]","[5.0, 10.0, 10.0]",True,23,25,False,False,10.0,False
1929,loss,"[9.0, 10.0]","[6.0, 10.0]",False,19,16,False,False,9.0,False
1930,win,"[10.0, 2.0]","[10.0, 1.0]",False,12,[21],False,True,10.0,False
1931,loss,"[10.0, 10.0]","[2.0, 10.0, 5.0]",True,20,17,False,False,10.0,False


In [101]:
win_loss_count = pd.DataFrame(
    df.groupby(["outcome"]).count()["dealer_hand"]).rename(
        columns = {"dealer_hand": "count"}
)
win_loss_count

,count
outcome,
loss,1166
push,114
win,653


In [103]:
dealer_card_results = df.groupby(
    ["dealer_card", "outcome"]).count()["dealer_hand"].unstack().fillna(0)
dealer_card_results

outcome,loss,push,win
dealer_card,,,
1.0,124,6,40
2.0,84,6,63
3.0,88,10,52
4.0,70,5,64
5.0,85,12,63
6.0,87,9,58
7.0,88,9,53
8.0,67,13,45
9.0,74,11,39


In [104]:
splits = list(filter(lambda x: len(x)==2, observations))
splits

[{0: {'outcome': 'loss',
   'dealer_hand': array([ 9.,  2., 10.]),
   'player_hand': [2.0, 9.0, 2.0],
   'double': True},
  1: {'outcome': 'loss',
   'dealer_hand': array([ 9.,  2., 10.]),
   'player_hand': [2.0, 8.0, 4.0],
   'double': True}},
 {0: {'outcome': 'loss',
   'dealer_hand': [9.0, 10.0],
   'player_hand': [1.0, 3.0, 1.0],
   'double': True},
  1: {'outcome': 'loss',
   'dealer_hand': [9.0, 10.0],
   'player_hand': [1.0, 1.0, 2.0],
   'double': True}},
 {0: {'outcome': 'loss',
   'dealer_hand': [6.0, 10.0],
   'player_hand': [4.0, 6.0, 10.0, 3.0],
   'double': False},
  1: {'outcome': 'loss',
   'dealer_hand': [6.0, 10.0],
   'player_hand': [4.0, 9.0],
   'double': False}},
 {0: {'outcome': 'loss',
   'dealer_hand': [7.0, 10.0],
   'player_hand': [6.0, 10.0, 8.0],
   'double': True},
  1: {'outcome': 'loss',
   'dealer_hand': [7.0, 10.0],
   'player_hand': [6.0, 9.0, 9.0],
   'double': False}},
 {0: {'outcome': 'loss',
   'dealer_hand': [10.0, 10.0],
   'player_hand': [8.0, 